In [1]:
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/research'

In [2]:
import os

os.chdir('../')
%pwd

'/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf'

#### Callbacks Entity

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class CallbacksConfig:
    root_dir: Path
    tensorboard_log_dir: Path
    model_checkpoint_filepath: Path

#### Configuration Manager - Callbacks

In [7]:
from cnn_classifier.constants import *
from cnn_classifier.utils.utilities import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
    
    def get_callbacks_config(self) -> CallbacksConfig:
        config = self.config.callbacks
        model_ckpt_dir = os.path.dirname(config.model_checkpoint_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_log_dir)
        ])

        callbacks_config = CallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_log_dir=Path(config.tensorboard_log_dir),
            model_checkpoint_filepath=Path(config.model_checkpoint_filepath)
        )

        return callbacks_config

#### Callbacks Component

In [ ]:
import tensorflow as tf
import time

class Callbacks:
    def __init__(self, config=CallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.model_checkpoint_filepath,
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

/Users/neehanth/Documents/Data Scientist/Machine Learning/chicken_disease_clf/cnn_venv/lib/python3.12/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


#### Callbacks Pipeline

In [9]:
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callbacks_config()
    prepare_callbacks = Callbacks(config=callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2025-12-22 11:09:38,086]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='config/config.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-22 11:09:38,089]: cnnClassiferLogger - INFO - utilities - yaml file '<_io.TextIOWrapper name='params.yaml' mode='r' encoding='UTF-8'> loaded successfully!'
[2025-12-22 11:09:38,089]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts
[2025-12-22 11:09:38,090]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts/callbacks/checkpoint_dir
[2025-12-22 11:09:38,090]: cnnClassiferLogger - INFO - utilities - Created directory at artifacts/callbacks/tensorboard_log_dir
